# Video Summarizer 모델 학습/평가 코드

In [1]:
import math
from data_loader import DataLoader
from model import c3d_model as YasuoNet
from trainer import Trainer
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.utils import class_weight

## 하이퍼파라미터 설정

In [2]:
dataset_dir = r'E:\Work\YasuoNet\data\dataset14_sl5_vsr3_vw64_vh64_asr11025'
ckpt_dir = 'checkpoints'
learning_rate = 1e-4
epochs = 3
batch_size = 500

## 데이터 로더 생성

In [3]:
data_loader = DataLoader(dataset_dir)
data_config = data_loader.get_metadata()['config']
video_frames = int(data_config['segment_length'] * data_config['video_sample_rate'])
video_width = data_config['video_width']
video_height = data_config['video_height']
video_channels = 3  # 향후 메타데이터에서 읽어오도록 수정

# epoch 당 배치 수
train_steps = data_loader.get_train_data_count() // batch_size
valid_steps = math.ceil(data_loader.get_valid_data_count() / batch_size)
test_steps = math.ceil(data_loader.get_test_data_count() / batch_size)
train_steps, valid_steps, test_steps

(14, 5, 5)

## 모델 생성

In [4]:
model = YasuoNet(video_frames, video_width, video_height, video_channels)
model.compile(Adam(learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

## 모델 학습

In [5]:
class_weights = class_weight.compute_class_weight('balanced', classes=[0, 1], y=data_loader.all_segment_df['label'].to_numpy())
class_weights = {i: v for i, v in enumerate(class_weights)}
class_weights

{0: 0.5496676341166558, 1: 5.533459000942507}

In [6]:
trainer = Trainer(model, ckpt_dir, learning_rate, epochs, class_weight)
trainer.train(
    data_loader.iter_train_batch_data(batch_size), train_steps,
    data_loader.iter_valid_batch_data(batch_size), valid_steps
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 14 steps, validate for 5 steps
Epoch 1/3
13/14 [==========================>...] - ETA: 27s - loss: 0.6668 - accuracy: 0.8812
Epoch 00001: val_loss improved from inf to 0.49288, saving model to checkpoints\ckpt-0001-0.4929.hdf5
14/14 [==============================] - 398s 28s/step - loss: 0.6592 - accuracy: 0.8833 - val_loss: 0.4929 - val_accuracy: 0.9237
Epoch 2/3
13/14 [==========================>...] - ETA: 27s - loss: 0.5166 - accuracy: 0.9100
Epoch 00002: val_loss improved from 0.49288 to 0.45432, saving model to checkpoints\ckpt-0002-0.4543.hdf5
14/14 [==============================] - 400s 29s/step - loss: 0.5147 - accuracy: 0.9100 - val_loss: 0.4543 - val_accuracy: 0.9237
Epoch 3/3
13/14 [==========================>...] - ETA: 27s - loss: 0.4693 - accuracy: 0.9100
Epoch 00003: val_loss improved from 0.45432 to 0.42756, saving model to checkpoints\ckpt-0003-0.4276.hdf5
14/14 [==============================] - 395s 28s/s

In [7]:
trainer.test(data_loader.iter_test_batch_data(batch_size), test_steps)

  ...
    to  
  ['...']
5/5 [==============================] - 13s 3s/step - loss: 0.4872 - accuracy: 0.8904


In [8]:
test_batch_iter = data_loader.iter_test_batch_data(batch_size, repeat=False)

result = []
for x, y in test_batch_iter:
    pred = model.predict(x)
    y_hat = (pred > 0.5) * 1

    result.append(np.vstack([y.squeeze(), y_hat.squeeze()]).T)
    
result = np.vstack(result)

In [9]:
len(result)

2227

In [10]:
result.sum(axis=0)

array([244,   0])